In [1]:
import ATS_Analysis as ats
import pandas as pd
import numpy as np

In [3]:
df = ats.get_data(['P110', 'P112'], 10)

['VentDB', 'VentDyssynchrony_db', 'local']
['BreathData_collection', 'LabData_collection', 'LogData_collection', 'PatientData_collection', 'RNData_collection', 'RTData_collection', 'TestData_collection', 'ValidatedData_collection', 'VentSettings_collection']
P110 95 RN_entry     33
patientID    33
RASS         33
dtype: int64 27
['VentDB', 'VentDyssynchrony_db', 'local']
['BreathData_collection', 'LabData_collection', 'LogData_collection', 'PatientData_collection', 'RNData_collection', 'RTData_collection', 'TestData_collection', 'ValidatedData_collection', 'VentSettings_collection']
P112 23 RN_entry     7
patientID    7
RASS         7
dtype: int64 7


In [5]:
df

,RASS,patientID,ds_rolling,tot_rolling,ds_freq,RASS,patientID,ds_rolling,tot_rolling,ds_freq
2014-10-16 16:30:00,-2,NaN,87.0,170,0.511765,NaN,NaN,NaN,NaN,NaN
2014-10-16 20:00:00,-2,NaN,108.0,176,0.613636,NaN,NaN,NaN,NaN,NaN
2014-10-17 00:00:00,-2,NaN,53.0,177,0.299435,NaN,NaN,NaN,NaN,NaN
2014-10-17 04:00:00,-2,NaN,96.0,180,0.533333,NaN,NaN,NaN,NaN,NaN
2014-10-17 07:45:00,-2,NaN,103.0,186,0.553763,NaN,NaN,NaN,NaN,NaN
2014-10-17 13:00:00,-2,NaN,73.0,180,0.405556,NaN,NaN,NaN,NaN,NaN
2014-10-17 20:00:00,-2,110,45.0,186,0.241935,NaN,NaN,NaN,NaN,NaN
2014-10-18 00:00:00,-2,NaN,115.0,237,0.485232,NaN,NaN,NaN,NaN,NaN
2014-10-18 04:00:00,-1,NaN,50.0,192,0.260417,NaN,NaN,NaN,NaN,NaN
2014-10-18 16:00:00,-3,NaN,162.0,229,0.707424,NaN,NaN,NaN,NaN,NaN
